### Variant effect predictions in from python

Main function for variants effect predictions is `score_variants`:

In [1]:
import kipoi_veff.snv_predict as sp
import os

In [2]:
sp.score_variants?

Signature: sp.score_variants(model, dl_args, input_vcf, output_vcf=None, output_writers=None, scores=['logit_ref', 'logit_alt', 'ref', 'alt', 'logit', 'diff'], score_kwargs=None, num_workers=0, batch_size=32, source='kipoi', seq_length=None, std_var_id=False, restriction_bed=None, return_predictions=False, model_outputs=None)
Docstring:
Score variants: annotate the vcf file using model predictions for the reference and alternative alleles

The functional elements that generate a score from a set of predictions for reference and
alternative allele are defined in the `scores` argument.

This function is the python version of the command-line call `score_variants` and is a convenience version
of the `predict_snvs` function:

Prediction of effects of SNV based on a VCF. If desired the VCF can be stored with the predicted values as
annotation. For a detailed description of the requirements in the yaml files please take a look at
the core `kipoi` documentation on how to write a `dataloader.y

In [3]:
sp.score_variants(model='Basset',
                  dl_args={'fasta_file': 'input/hg19.chr22.fa'},
                  input_vcf='input/clinvar_20180429.pathogenic.chr22.vcf.gz',
                  output_vcf='/tmp/py-annotated.vcf')

Using downloaded and verified file: /home/avsec/.kipoi/models/Basset/downloaded/model_files/weights/4878981d84499eb575abd0f3b45570d3


100%|██████████| 19/19 [00:18<00:00,  1.18it/s]


In [4]:
!head /tmp/py-annotated.vcf

##fileformat=VCFv4.1
##fileDate=2018-04-29
##source=ClinVar
##reference=GRCh37
##ID=<Description="ClinVar Variation ID">
##INFO=<ID=AF_ESP,Number=1,Type=Float,Description="allele frequencies from GO-ESP">
##INFO=<ID=AF_EXAC,Number=1,Type=Float,Description="allele frequencies from ExAC">
##INFO=<ID=AF_TGP,Number=1,Type=Float,Description="allele frequencies from TGP">
##INFO=<ID=ALLELEID,Number=1,Type=Integer,Description="the ClinVar Allele ID">
##INFO=<ID=CLNDN,Number=.,Type=String,Description="ClinVar's preferred disease name for the concept specified by disease identifiers in CLNDISDB">


### Using BatchWriters from Kipoi instead of the VCF output

In [5]:
from kipoi_veff.utils.io import SyncBatchWriter
from kipoi.writers import HDF5BatchWriter

Via SyncBatchWriter you can use the batch writers from Kipoi

In [6]:
!rm /tmp/annotated.vcf.h5

In [7]:
from tempfile import mkdtemp

In [8]:
h5_tmp = os.path.join(mkdtemp(), 'output.h5')

In [9]:
writer = SyncBatchWriter(HDF5BatchWriter(h5_tmp))

In [10]:
sp.score_variants(model='Basset',
                  dl_args={'fasta_file': 'input/hg19.chr22.fa'},
                  input_vcf='input/clinvar_20180429.pathogenic.chr22.vcf.gz',
                  output_writers=[writer])

  0%|          | 0/19 [00:00<?, ?it/s]

Using downloaded and verified file: /home/avsec/.kipoi/models/Basset/downloaded/model_files/weights/4878981d84499eb575abd0f3b45570d3


100%|██████████| 19/19 [00:18<00:00,  1.28it/s]


In [11]:
from kipoi.readers import HDF5Reader

In [12]:
r = HDF5Reader(h5_tmp)
r.open()

In [13]:
r.ls()[:20]

[('/line_idx', <HDF5 dataset "line_idx": shape (582,), type "|O">),
 ('/preds/alt/8988T', <HDF5 dataset "8988T": shape (582,), type "<f4">),
 ('/preds/alt/A549', <HDF5 dataset "A549": shape (582,), type "<f4">),
 ('/preds/alt/ADRL_GLND_FET',
  <HDF5 dataset "ADRL_GLND_FET": shape (582,), type "<f4">),
 ('/preds/alt/AG04449', <HDF5 dataset "AG04449": shape (582,), type "<f4">),
 ('/preds/alt/AG04450', <HDF5 dataset "AG04450": shape (582,), type "<f4">),
 ('/preds/alt/AG09309', <HDF5 dataset "AG09309": shape (582,), type "<f4">),
 ('/preds/alt/AG09319', <HDF5 dataset "AG09319": shape (582,), type "<f4">),
 ('/preds/alt/AG10803', <HDF5 dataset "AG10803": shape (582,), type "<f4">),
 ('/preds/alt/AoAF', <HDF5 dataset "AoAF": shape (582,), type "<f4">),
 ('/preds/alt/AoSMC', <HDF5 dataset "AoSMC": shape (582,), type "<f4">),
 ('/preds/alt/BE2_C', <HDF5 dataset "BE2_C": shape (582,), type "<f4">),
 ('/preds/alt/BJ', <HDF5 dataset "BJ": shape (582,), type "<f4">),
 ('/preds/alt/BLD_CD14_PC',


In [14]:
out = r.load_all()

In [15]:
out['preds']['diff']['A549']

array([ 0.0149, -0.0013, -0.0008, -0.0009, -0.0008, -0.0019,  0.0005, -0.0028,  0.0074, -0.0084,
        0.0035,  0.0015,  0.0028,  0.0035,  0.0049,  0.0022, -0.0013,  0.0054,  0.0148,  0.0364,
       ...,  0.0016, -0.0008,  0.0047,  0.0018,  0.0035,  0.0021,  0.0004,  0.0024, -0.0051,
       -0.0229, -0.0108, -0.0122, -0.0045,  0.021 ,  0.0102,  0.0345,  0.0003,  0.0076, -0.0071,
        0.0066], dtype=float32)